In [1]:
import numpy as np
from asdf import AsdfFile
from astropy.io import fits
from astropy import wcs as astwcs
from gwcs import wcs

from jwst import datamodels
from jwst.assign_wcs import nirspec
from jwst.transforms import models

In [2]:
# These are the CV3 files
# It uses the new ifupost and ote files
refs = {'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf',
        'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf',
        'disperser': '/grp/crds/cache/references/jwst/jwst_nirspec_disperser_0035.asdf',
        'distortion': 'N/A',
        'filteroffset': 'N/A',
        'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf',
        'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf',
        'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf',
        'ifupost': 'ifupost.asdf',
        'ifuslicer': '/grp/crds/cache/references/jwst/jwst_nirspec_ifuslicer_0003.asdf',
        'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf',
        'ote': 'ote.asdf',
        'regions': 'N/A',
        'specwcs': 'N/A',
        'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf'}


In [3]:
wcs_kw = {'wcsaxes': 2, 'ra_ref': 165, 'dec_ref': 54,
          'v2_ref': -8.3942412, 'v3_ref': -5.3123744, 'roll_ref': 37,
          'crpix1': 1024, 'crpix2': 1024,
          'cdelt1': .08, 'cdelt2': .08,
          'ctype1': 'RA---TAN', 'ctype2': 'DEC--TAN',
          'pc1_1': 1, 'pc1_2': 0, 'pc2_1': 0, 'pc2_2': 1
          }

slit_fields_num = ["shutter_id", "xcen", "ycen",
                   "ymin", "ymax", "quadrant", "source_id",
                   "stellarity", "source_xpos", "source_ypos"]


slit_fields_str = ["name", "shutter_state", "source_name", "source_alias"]


In [4]:
def create_hdul(detector='NRS1'):
    """
    Create a fits HDUList instance.
    """
    hdul = fits.HDUList()
    phdu = fits.PrimaryHDU()
    phdu.header['instrume'] = 'NIRSPEC'
    phdu.header['detector'] = detector
    phdu.header['time-obs'] = '8:59:37'
    phdu.header['date-obs'] = '2016-09-05'

    scihdu = fits.ImageHDU()
    scihdu.header['EXTNAME'] = "SCI"
    for item in wcs_kw.items():
        scihdu.header[item[0]] = item[1]
    hdul.append(phdu)
    hdul.append(scihdu)
    return hdul

In [5]:
def create_nirspec_ifu_file(filter='F290LP', grating='G395H', lamp='N/A', detector='NRS1'):
    image = create_hdul(detector)
    image[0].header['exp_type'] = 'NRS_IFU'
    image[0].header['filter'] = filter
    image[0].header['grating'] = grating
    image[1].header['crval3'] = 0
    image[1].header['wcsaxes'] = 3
    image[1].header['ctype3'] = 'WAVE'
    image[0].header['lamp'] = lamp
    image[0].header['GWA_XTIL'] = 0.35986012
    image[0].header['GWA_YTIL'] = 0.13448857
    return image

In [6]:
!ulimit -n 1024
hdul = create_nirspec_ifu_file(grating='G395H', filter='F290LP',  detector='NRS1')
im = datamodels.ImageModel(hdul)

pipeline = nirspec.create_pipeline(im, refs)
w = wcs.WCS(pipeline)
im.meta.wcs = w

2018-06-19 14:28:32,430 - stpipe - INFO - gwa_ytilt is 0.13448857 deg
2018-06-19 14:28:32,431 - stpipe - INFO - gwa_xtilt is 0.35986012 deg
2018-06-19 14:28:32,432 - stpipe - INFO - theta_y correction: -0.041645342310905185 deg
2018-06-19 14:28:32,433 - stpipe - INFO - theta_x correction: 0.0 deg
2018-06-19 14:28:45,265 - stpipe - INFO - Created a NIRSPEC nrs_ifu pipeline with references {'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf', 'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf', 'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf', 'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf', 'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf', 'regions': 'N/A', 'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf', 'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf', 'ote': 'ote.asdf', 'ifuslicer': '/grp/crds/cache/refe

In [7]:
# Setup the test
slitx = [0] * 5
slity = [-.5, -.25, 0, .25, .5]
wave_range = [2.87e-06, 5.27e-06]
lam = np.array([2.9, 3.39, 3.88, 4.37, 5]) * 10**-6

order, wrange = nirspec.get_spectral_order_wrange(im, refs['wavelengthrange'])
im.meta.wcsinfo.sporder = order
im.meta.wcsinfo.waverange_start = wrange[0]
im.meta.wcsinfo.waverange_end = wrange[1]
# Use slice 0
slit_wcs = nirspec.nrs_wcs_set_input(im, 0)


In [8]:
# Slit to MSA entrance
# This includes the Slicer transform and the IFUFORE transform
slit2msa = slit_wcs.get_transform('slit_frame', 'msa_frame')
msax, msay, _= slit2msa(slitx, slity, lam)
print('slitx: ', slitx )
print('slity: ', slity)
print('msax: ', msax)
print('msay: ', msay)


slitx:  [0, 0, 0, 0, 0]
slity:  [-0.5, -0.25, 0, 0.25, 0.5]
msax:  [ 0.04131163  0.04131081  0.04131053  0.04131081  0.04131163]
msay:  [ -6.16116144e-04  -3.08195439e-04   3.09418172e-09   3.08202387e-04
   6.16125124e-04]


In [9]:
# Slicer
slit2slicer = slit_wcs.get_transform('slit_frame', 'slicer')
x_slicer, y_slicer, _ = slit2slicer(slitx, slity, lam)
print('x_slicer: ', x_slicer)
print('y_slicer:', y_slicer)

x_slicer:  [ 0.0004  0.0004  0.0004  0.0004  0.0004]
y_slicer: [-0.006 -0.003  0.     0.003  0.006]


In [10]:
# MSA exit
# Applies the IFUPOST transform to coordinates at the Slicer
ifupost = datamodels.IFUPostModel('ifupost.asdf')
ifupost_transform = nirspec._create_ifupost_transform(ifupost.slice_0)
x_msa_exit, y_msa_exit = ifupost_transform(x_slicer, y_slicer, lam)
ifupost.close()
print('x_msa_exit: ', x_msa_exit)
print('y_msa_exit: ', y_msa_exit)

x_msa_exit:  [ 0.04862197  0.04861941  0.04861821  0.04861838  0.04861992]
y_msa_exit:  [ 0.00794002  0.00824178  0.00854341  0.00884478  0.00914578]


In [11]:
# Coordinates at Collimator exit
# Applies the Collimator forward transform to coordinates at the MSA exit
col = datamodels.open(refs['collimator'])
colx, coly = col.model.inverse(x_msa_exit, y_msa_exit)
col.close()
print('x_collimator_exit', colx)
print('y_collimator_exit', coly)

x_collimator_exit [ 0.07443997  0.07443401  0.07443022  0.07442859  0.07442913]
y_collimator_exit [-0.28242043 -0.28192622 -0.28143237 -0.28093908 -0.28044655]


In [12]:
# Slit to GWA entrance
# applies the Collimator forward, Unitless to Directional and 3D Rotation to MSA exit coordinates
disp = datamodels.DisperserModel(refs['disperser'])
disperser = nirspec.correct_tilt(disp, im.meta.instrument.gwa_xtilt, im.meta.instrument.gwa_ytilt)
collimator2gwa = nirspec.collimator_to_gwa(refs, disperser)
x_gwa_in, y_gwa_in, z_gwa_in = collimator2gwa(x_msa_exit, y_msa_exit)
disp.close()
print('x_gwa_entrance:' , x_gwa_in)
print('y_gwa_entrance:' , y_gwa_in)
print('z_gwa_entrance:' , z_gwa_in)

2018-06-19 14:28:47,983 - stpipe - INFO - gwa_ytilt is 0.13448857 deg
2018-06-19 14:28:47,984 - stpipe - INFO - gwa_xtilt is 0.35986012 deg
2018-06-19 14:28:47,985 - stpipe - INFO - theta_y correction: -0.041645342310905185 deg
2018-06-19 14:28:47,986 - stpipe - INFO - theta_x correction: 0.0 deg


x_gwa_entrance: [ 0.22067237  0.22069163  0.22071285  0.22073601  0.22076113]
y_gwa_entrance: [-0.27015622 -0.26971665 -0.26927718 -0.26883799 -0.26839925]
z_gwa_entrance: [ 0.93718692  0.93730899  0.93743034  0.93755093  0.93767071]


In [13]:
# Slit to GWA out
# Runs slit--> slicer --> msa_exit --> collimator --> dircos --> rotation --> angle_from_grating equation
slit2gwa = slit_wcs.get_transform('slit_frame', 'gwa')
x_gwa_out, y_gwa_out, z_gwa_out = slit2gwa(slitx, slity, lam)
print('x_gwa_exit:' , x_gwa_out)
print('y_gwa_exit:' , y_gwa_out)
print('z_gwa_exit:' , z_gwa_out)

x_gwa_exit: [ 0.03861286  0.08240387  0.12619291  0.16998001  0.22628237]
y_gwa_exit: [ 0.27015622  0.26971665  0.26927718  0.26883799  0.26839925]
z_gwa_exit: [ 0.96204192  0.95940739  0.95475921  0.948068    0.93635364]


In [14]:
# CAMERA entrance (assuming direction is from sky to detector)
angles = [disperser['theta_x'], disperser['theta_y'],
          disperser['theta_z'], disperser['tilt_y']]
rotation = models.Rotation3DToGWA(angles, axes_order="xyzy", name='rotation')
dircos2unitless = models.DirCos2Unitless()

gwa2cam = rotation.inverse | dircos2unitless
x_camera_entrance, y_camera_entrance = gwa2cam(x_gwa_out, y_gwa_out, z_gwa_out)
print('x_camera_entrance:' , x_camera_entrance)
print('y_camera_entrance:' , y_camera_entrance)

x_camera_entrance: [-0.11311156 -0.06716992 -0.02134772  0.02464534  0.08453302]
y_camera_entrance: [ 0.28285903  0.28083671  0.27940655  0.27856425  0.27849854]


In [15]:
# at FPA
camera = datamodels.CameraModel(refs['camera'])
x_fpa, y_fpa = camera.model.inverse(x_camera_entrance, y_camera_entrance)
camera.close()
print('x_fpa: ', x_fpa )
print('y_fpa: ' , y_fpa)

x_fpa:  [-0.03260156 -0.0196373  -0.00661481  0.00648947  0.02349652]
y_fpa:  [-0.00354315 -0.0038227  -0.00402751 -0.00416094 -0.00418796]


In [16]:
# at SCA
slit2sca = slit_wcs.get_transform('slit_frame', 'sca')
x_sca_nrs1, y_sca_nrs1 = slit2sca(slitx, slity, lam)

# At NRS2
fpa = datamodels.FPAModel(refs['fpa'])
x_sca_nrs2, y_sca_nrs2 = fpa.nrs2_model.inverse(x_fpa, y_fpa)
fpa.close()
print('x_sca1: ', x_sca_nrs1)
print('y_sca1: ' , y_sca_nrs1)
print('x_sca2: ', x_sca_nrs2 )
print('y_sca2: ' , y_sca_nrs2)

x_sca1:  [  309.40456252  1029.64117666  1753.11275396  2481.12822002  3425.96453057]
y_sca1:  [ 826.65806896  811.1275858   799.74961365  792.33653939  790.83550167]
x_sca2:  [ 3931.81779465  3211.58141255  2488.11000527  1760.09465002   815.25836199]
y_sca2:  [ 1220.24684926  1235.78808985  1247.17686774  1254.60081562  1256.11596538]


In [17]:
# at oteip
# Goes through slicer, ifufore, and fore transforms
slit2oteip = slit_wcs.get_transform('slit_frame', 'oteip')
x_oteip, y_oteip, _ = slit2oteip(slitx, slity, lam)
print('x_oteip: ', x_oteip)
print('y_oteip: ' , y_oteip)

x_oteip:  [ 0.05010543  0.05044043  0.05077637  0.05111296  0.05145047]
y_oteip:  [-0.04489819 -0.04451235 -0.04412679 -0.04374186 -0.04335772]


In [18]:
# at v2, v3 [in arcsec]
slit2v23 = slit_wcs.get_transform('slit_frame', 'v2v3')
v2, v3, _ = slit2v23(slitx, slity, lam)
v2 /= 3600
v3 /= 3600
print('v2: ', v2)
print('v3: ' , v3)

v2:  [ 0.08365329  0.08350689  0.08336007  0.08321298  0.08306549]
v3:  [-0.1387331  -0.13856511 -0.13839724 -0.13822965 -0.1380624 ]


In [19]:
# Save results to an asdf file

fa = AsdfFile()
fa.tree['slitx'] = list(slitx)
fa.tree['slity'] = list(slity)
fa.tree['lam'] = list(lam)
fa.tree['msax'] = list(msax)
fa.tree['msay'] = list(msay)
fa.tree['x_collimator_exit'] = list(colx)
fa.tree['y_collimator_exit'] = list(coly)
fa.tree['x_gwa_entrance'] = list(x_gwa_in)
fa.tree['y_gwa_entrance'] = list(y_gwa_in)
fa.tree['z_gwa_entrance'] = list(z_gwa_in)
fa.tree['x_gwa_exit'] = list(x_gwa_out)
fa.tree['y_gwa_exit'] = list(y_gwa_out)
fa.tree['z_gwa_exit'] = list(z_gwa_out)
fa.tree['x_camera_entrance'] = list(x_camera_entrance)
fa.tree['y_camera_entrance'] = list(y_camera_entrance)
fa.tree['x_fpa'] = list(x_fpa)
fa.tree['y_fpa'] = list(y_fpa)
fa.tree['x_sca_nrs1'] = list(x_sca_nrs1)
fa.tree['y_sca_nrs1'] = list(y_sca_nrs1)
fa.tree['x_sca_nrs2'] = list(x_sca_nrs2)
fa.tree['y_sca_nrs2'] = list(y_sca_nrs2)
fa.tree['x_oteip'] = list(x_oteip)
fa.tree['y_oteip'] = list(y_oteip)
fa.tree['v2'] = list(v2)
fa.tree['v3'] = list(v3)
fa.tree['x_slicer'] = list(x_slicer)
fa.tree['y_slicer'] = list(y_slicer)
fa.tree['x_msa_exit'] = list(x_msa_exit)
fa.tree['y_msa_exit'] = list(y_msa_exit)
fa.write_to("ifu_grating_functional.asdf", all_array_storage="internal")